# Developpement Modèle depuis la bdd 

In [1]:
import psycopg2
import pandas as pd

# Database configuration
DB_CONFIG = {
    "host": "localhost",
    "database": "ClashOfRse",
    "user": "your_username",
    "password": "your_password",
}

def connect_to_db():
    """Connect to the PostgreSQL database."""
    return psycopg2.connect(**DB_CONFIG)

def fetch_all_tables_data():
    """Fetch all tables from the database and store them in a single DataFrame."""
    try:
        connection = connect_to_db()
        cursor = connection.cursor()

        # Fetch all table names
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
        tables = [row[0] for row in cursor.fetchall()]

        if not tables:
            raise ValueError("No tables found in the database.")

        # Dictionary to store each table's data as a DataFrame
        table_dataframes = {}

        # Fetch data from each table
        for table in tables:
            query = f"SELECT * FROM {table}"
            table_df = pd.read_sql_query(query, connection)
            table_dataframes[table] = table_df

        # Combine all dataframes into one
        combined_df = pd.concat(
            [
                df.assign(table_name=table)  # Add a column indicating the table name
                for table, df in table_dataframes.items()
            ],
            ignore_index=True
        )

        print("Dataframe shape:", combined_df.shape)
        print("Columns:", combined_df.columns)

        return combined_df

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

if __name__ == "__main__":
    df = fetch_all_tables_data()
    print(df.head())

C:\Users\Yann\AppData\Local\Temp\ipykernel_13764\4023745716.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_df = pd.read_sql_query(query, connection)


Dataframe shape: (141327, 34)
Columns: Index(['roleid', 'nom', 'table_name', 'profilid', 'prenom', 'datedenaissance',
       'email', 'entreprise', 'image', 'distancetravailmaison',
       'progressionid', 'defiid', 'score', 'description', 'objectif',
       'begesid', 'datedebut', 'datefin', 'co2total', 'globalrank',
       'transportid', 'co2parkm', 'transportemissionid', 'co2emistransport',
       'rank', 'chauffageid', 'co2parm', 'chauffageemissionid',
       'co2emischauffage', 'alimentid', 'co2par100g', 'massekg',
       'alimentationemissionid', 'co2emisalimentation'],
      dtype='object')
   roleid      nom table_name  profilid    prenom datedenaissance  \
0     1.0    Admin       role       NaN       NaN             NaN   
1     2.0     User       role       NaN       NaN             NaN   
2     3.0  Manager       role       NaN       NaN             NaN   
3     4.0  Employé       role       NaN       NaN             NaN   
4     4.0   User_1     profil       1.0  Prenom_1 

In [2]:
# Compter les valeurs manquantes (NaN) par colonne
nan_counts = df.isna().sum()

# Afficher les résultats
print("Nombre de valeurs manquantes (NaN) par colonne :")
print(nan_counts)

Nombre de valeurs manquantes (NaN) par colonne :
roleid                    141223
nom                       141126
table_name                     0
profilid                  138849
prenom                    141227
datedenaissance           141227
email                     141227
entreprise                141227
image                     141327
distancetravailmaison     141227
progressionid             140349
defiid                    140299
score                     140349
description               141277
objectif                  141277
begesid                     1179
datedebut                   1179
datefin                     1179
co2total                  139927
globalrank                139927
transportid                56930
co2parkm                  141300
transportemissionid        56957
co2emistransport           56957
rank                        2579
chauffageid                99119
co2parm                   141319
chauffageemissionid        99127
co2emischauffage           

In [5]:
import psycopg2
import pandas as pd

# Database configuration
DB_CONFIG = {
    "host": "localhost",
    "database": "ClashOfRse",
    "user": "your_username",
    "password": "your_password",
}

def connect_to_db():
    """Connect to the PostgreSQL database."""
    return psycopg2.connect(**DB_CONFIG)

def fetch_table_data(table_name, connection):
    """Fetch data from a specific table."""
    query = f"SELECT * FROM {table_name}"
    return pd.read_sql_query(query, connection)

def identify_missing_data():
    """Identify missing data in all tables and propose corrections."""
    try:
        connection = connect_to_db()
        tables = ["Role", "Profil", "Transport", "Aliment", "Chauffage", "TransportEmission",
                  "AlimentationEmission", "ChauffageEmission", "BEGES", "Defi", "ProgressionDefi"]
        missing_data_summary = {}

        for table in tables:
            print(f"Analyzing table: {table}")
            df = fetch_table_data(table, connection)
            missing_counts = df.isnull().sum()

            # Log missing counts
            missing_data_summary[table] = missing_counts[missing_counts > 0].to_dict()

        # Print summary of missing data
        print("\nMissing Data Summary:")
        for table, missing_columns in missing_data_summary.items():
            if missing_columns:
                print(f"- {table}:")
                for column, count in missing_columns.items():
                    print(f"    {column}: {count} missing values")

        return missing_data_summary

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if connection:
            connection.close()

def propose_corrections(missing_data_summary):
    """Propose corrections for missing data."""
    corrections = {}

    for table, missing_columns in missing_data_summary.items():
        for column in missing_columns:
            if "Id" in column:
                corrections[(table, column)] = "Ensure referenced table is populated and foreign keys are valid."
            elif "Nom" in column or "Prenom" in column:
                corrections[(table, column)] = "Generate random names or use placeholders."
            elif "CO2" in column:
                corrections[(table, column)] = "Impute with average or default CO2 values."
            elif "Date" in column:
                corrections[(table, column)] = "Fill with default dates or approximate ranges."
            elif "Score" in column or "Rank" in column:
                corrections[(table, column)] = "Impute with median or random valid scores/ranks."
            else:
                corrections[(table, column)] = "Analyze and determine logical imputation values."

    # Print corrections
    print("\nProposed Corrections:")
    for (table, column), correction in corrections.items():
        print(f"- {table}.{column}: {correction}")

    return corrections

if __name__ == "__main__":
    missing_data_summary = identify_missing_data()
    corrections = propose_corrections(missing_data_summary)

Analyzing table: Role
Analyzing table: Profil
Analyzing table: Transport
Analyzing table: Aliment
Analyzing table: Chauffage
Analyzing table: TransportEmission
Analyzing table: AlimentationEmission
Analyzing table: ChauffageEmission


C:\Users\Yann\AppData\Local\Temp\ipykernel_13764\853424244.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, connection)


Analyzing table: BEGES
Analyzing table: Defi
Analyzing table: ProgressionDefi

Missing Data Summary:

Proposed Corrections:


In [9]:
import pandas as pd
from sqlalchemy import create_engine
import logging
from typing import Dict

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s]: %(message)s")

# Database configuration
DB_CONFIG = {
    "host": "localhost",
    "database": "ClashOfRse",
    "user": "your_username",
    "password": "your_password",
}

def connect_to_db() -> create_engine:
    """Connect to the PostgreSQL database using SQLAlchemy."""
    try:
        db_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
        engine = create_engine(db_url)
        # Test the connection
        engine.connect()
        logging.info("Successfully connected to the database.")
        return engine
    except Exception as e:
        logging.error(f"Failed to connect to the database: {e}")
        raise

def fetch_table_data(table_name: str, engine: create_engine) -> pd.DataFrame:
    """Fetch data from a specific table."""
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, con=engine)
        if df.empty:
            logging.warning(f"Table '{table_name}' is empty.")
        return df
    except Exception as e:
        logging.error(f"Failed to fetch data from table '{table_name}': {e}")
        raise

def identify_missing_data(engine: create_engine) -> Dict[str, Dict[str, int]]:
    """Identify missing data in all tables."""
    tables = ["Role", "Profil", "Transport", "Aliment", "Chauffage", "TransportEmission",
              "AlimentationEmission", "ChauffageEmission", "BEGES", "Defi", "ProgressionDefi"]
    missing_data_summary = {}

    for table in tables:
        try:
            logging.info(f"Analyzing table: {table}")
            df = fetch_table_data(table, engine)
            missing_counts = df.isnull().sum()
            missing_data_summary[table] = missing_counts[missing_counts > 0].to_dict()
        except Exception as e:
            logging.error(f"Error analyzing table '{table}': {e}")
    
    # Log missing data summary
    logging.info("\nMissing Data Summary:")
    for table, missing_columns in missing_data_summary.items():
        if missing_columns:
            logging.info(f"- {table}:")
            for column, count in missing_columns.items():
                logging.info(f"    {column}: {count} missing values")
        else:
            logging.info(f"- {table}: No missing values.")
    
    return missing_data_summary

def propose_corrections(missing_data_summary: Dict[str, Dict[str, int]]) -> Dict[tuple, str]:
    """Propose corrections for missing data."""
    corrections = {}

    for table, missing_columns in missing_data_summary.items():
        for column in missing_columns:
            if "Id" in column:
                corrections[(table, column)] = "Ensure referenced table is populated and foreign keys are valid."
            elif "Nom" in column or "Prenom" in column:
                corrections[(table, column)] = "Generate random names or use placeholders."
            elif "CO2" in column:
                corrections[(table, column)] = "Impute with average or default CO2 values."
            elif "Date" in column:
                corrections[(table, column)] = "Fill with default dates or approximate ranges."
            elif "Score" in column or "Rank" in column:
                corrections[(table, column)] = "Impute with median or random valid scores/ranks."
            else:
                corrections[(table, column)] = "Analyze and determine logical imputation values."

    # Log proposed corrections
    logging.info("\nProposed Corrections:")
    for (table, column), correction in corrections.items():
        logging.info(f"- {table}.{column}: {correction}")

    return corrections

if __name__ == "__main__":
    try:
        # Connect to the database
        engine = connect_to_db()

        # Identify missing data
        missing_data_summary = identify_missing_data(engine)

        # Propose corrections
        corrections = propose_corrections(missing_data_summary)

        # Optionally save results to a file
        pd.DataFrame.from_dict(missing_data_summary, orient='index').to_csv("missing_data_summary.csv")
        logging.info("Missing data summary saved to 'missing_data_summary.csv'.")

    except Exception as e:
        logging.error(f"An error occurred during execution: {e}")


2025-01-21 01:21:16,022 [INFO]: Successfully connected to the database.
2025-01-21 01:21:16,023 [INFO]: Analyzing table: Role
2025-01-21 01:21:16,029 [INFO]: Analyzing table: Profil
2025-01-21 01:21:16,034 [INFO]: Analyzing table: Transport
2025-01-21 01:21:16,037 [INFO]: Analyzing table: Aliment
2025-01-21 01:21:16,043 [INFO]: Analyzing table: Chauffage
2025-01-21 01:21:16,049 [INFO]: Analyzing table: TransportEmission
2025-01-21 01:21:16,407 [INFO]: Analyzing table: AlimentationEmission
2025-01-21 01:21:16,474 [INFO]: Analyzing table: ChauffageEmission
2025-01-21 01:21:16,616 [INFO]: Analyzing table: BEGES
2025-01-21 01:21:16,624 [INFO]: Analyzing table: Defi
2025-01-21 01:21:16,626 [INFO]: Analyzing table: ProgressionDefi
2025-01-21 01:21:16,631 [INFO]: 
Missing Data Summary:
2025-01-21 01:21:16,631 [INFO]: - Role: No missing values.
2025-01-21 01:21:16,632 [INFO]: - Profil: No missing values.
2025-01-21 01:21:16,633 [INFO]: - Transport: No missing values.
2025-01-21 01:21:16,634 [I

In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Database configuration
DB_CONFIG = {
    "host": "localhost",
    "database": "ClashOfRse",
    "user": "your_username",
    "password": "your_password",
}

def connect_to_db():
    """Connect to the PostgreSQL database using SQLAlchemy."""
    db_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
    return create_engine(db_url)

def fetch_and_export_table(table_name, engine):
    """Fetch data from a specific table and export it to a CSV file."""
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, con=engine)
        file_name = f"{table_name}.csv"
        df.to_csv(file_name, index=False)
        print(f"Table '{table_name}' exported to '{file_name}'.")
    except Exception as e:
        print(f"An error occurred while exporting table '{table_name}': {e}")

def export_all_tables():
    """Export all tables to separate CSV files."""
    tables = [
        "Role", "Profil", "Transport", "Aliment", "Chauffage",
        "TransportEmission", "AlimentationEmission", "ChauffageEmission",
        "BEGES", "Defi", "ProgressionDefi"
    ]
    try:
        engine = connect_to_db()
        for table in tables:
            fetch_and_export_table(table, engine)
        print("All tables have been successfully exported.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    export_all_tables()


Table 'Role' exported to 'Role.csv'.
Table 'Profil' exported to 'Profil.csv'.
Table 'Transport' exported to 'Transport.csv'.
Table 'Aliment' exported to 'Aliment.csv'.
Table 'Chauffage' exported to 'Chauffage.csv'.
Table 'TransportEmission' exported to 'TransportEmission.csv'.
Table 'AlimentationEmission' exported to 'AlimentationEmission.csv'.
Table 'ChauffageEmission' exported to 'ChauffageEmission.csv'.
Table 'BEGES' exported to 'BEGES.csv'.
Table 'Defi' exported to 'Defi.csv'.
Table 'ProgressionDefi' exported to 'ProgressionDefi.csv'.
All tables have been successfully exported.


In [14]:
import pandas as pd
from sqlalchemy import create_engine

# Database configuration
DB_CONFIG = {
    "host": "localhost",
    "database": "ClashOfRse",
    "user": "your_username",
    "password": "your_password",
}

def connect_to_db():
    """Connect to the PostgreSQL database using SQLAlchemy."""
    db_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
    return create_engine(db_url)

def analyze_user_data(profil_id):
    """Analyze data for a specific user (ProfilId)."""
    engine = connect_to_db()
    
    # Fetch profile details
    profile_query = f"SELECT * FROM Profil WHERE ProfilId = {profil_id}"
    profile = pd.read_sql_query(profile_query, con=engine)
    
    # Fetch BEGES records
    beges_query = f"SELECT * FROM BEGES WHERE ProfilId = {profil_id}"
    beges = pd.read_sql_query(beges_query, con=engine)
    
    # Fetch emissions data linked to BEGES
    transport_query = f"""
        SELECT te.* 
        FROM TransportEmission te 
        JOIN BEGES b ON te.BEGESId = b.BEGESId
        WHERE b.ProfilId = {profil_id}
    """
    transport_emissions = pd.read_sql_query(transport_query, con=engine)
    
    alimentation_query = f"""
        SELECT ae.* 
        FROM AlimentationEmission ae 
        JOIN BEGES b ON ae.BEGESId = b.BEGESId
        WHERE b.ProfilId = {profil_id}
    """
    alimentation_emissions = pd.read_sql_query(alimentation_query, con=engine)
    
    chauffage_query = f"""
        SELECT ce.* 
        FROM ChauffageEmission ce 
        JOIN BEGES b ON ce.BEGESId = b.BEGESId
        WHERE b.ProfilId = {profil_id}
    """
    chauffage_emissions = pd.read_sql_query(chauffage_query, con=engine)
    
    # Summarize emissions data
    total_transport_co2 = transport_emissions['co2emistransport'].sum()
    total_alimentation_co2 = alimentation_emissions['co2emisalimentation'].sum() if 'co2emisalimentation' in alimentation_emissions else 0
    total_chauffage_co2 = chauffage_emissions['co2emischauffage'].sum() if 'co2emischauffage' in chauffage_emissions else 0
    total_emissions = total_transport_co2 + total_alimentation_co2 + total_chauffage_co2

    # Debug: Log intermediate results
    print("\n--- Debugging Emissions Data ---")
    print("Transport Emissions Total:", total_transport_co2)
    print("Alimentation Emissions Total:", total_alimentation_co2)
    print("Heating Emissions Total:", total_chauffage_co2)
    print("Total Emissions:", total_emissions)
    
    # Print summaries
    print("\n--- Profile Details ---")
    print(profile)
    
    print("\n--- BEGES Records ---")
    print(beges)
    
    print("\n--- Emissions Summary ---")
    print(f"Transport CO2: {total_transport_co2} kg")
    print(f"Alimentation CO2: {total_alimentation_co2} kg")
    print(f"Chauffage CO2: {total_chauffage_co2} kg")
    print(f"Total CO2 Emissions: {total_emissions} kg")
    
    # Fetch challenge progression
    progression_query = f"""
        SELECT pd.*, d.Nom AS DefiNom, d.Description, d.Objectif
        FROM ProgressionDefi pd
        JOIN Defi d ON pd.DefiId = d.DefiId
        WHERE pd.ProfilId = {profil_id}
    """
    progression = pd.read_sql_query(progression_query, con=engine)
    print("\n--- Challenge Progression ---")
    print(progression)
    
    return {
        "profile": profile,
        "beges": beges,
        "transport_emissions": transport_emissions,
        "alimentation_emissions": alimentation_emissions,
        "chauffage_emissions": chauffage_emissions,
        "progression": progression
    }


if __name__ == "__main__":
    # Analyze data for a specific user (replace '1' with the desired ProfilId)
    user_data = analyze_user_data(1)



--- Debugging Emissions Data ---
Transport Emissions Total: 23865.87256959342
Alimentation Emissions Total: 1324.0329751981849
Heating Emissions Total: 9723.713702753625
Total Emissions: 34913.619247545226

--- Profile Details ---
   profilid  roleid     nom    prenom datedenaissance              email  \
0         1       4  User_1  Prenom_1      1971-03-07  user1@example.com   

     entreprise                                  image  distancetravailmaison  
0  Entreprise_6  https://example.com/default-image.png               4.510607  

--- BEGES Records ---
    begesid  profilid   datedebut     datefin     co2total  globalrank
0        84         1  2023-02-01  2023-03-03   868.275487           3
1        94         1  2023-05-14  2023-06-13   377.990205           3
2       115         1  2023-03-07  2023-04-06   552.838263           3
3       201         1  2023-01-01  2023-01-30   548.827636           3
4       202         1  2023-01-31  2023-03-01   630.396838           3
5     

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import create_engine

# Database configuration
DB_CONFIG = {
    "host": "localhost",
    "database": "ClashOfRse",
    "user": "your_username",
    "password": "your_password",
}

def connect_to_db():
    """Connect to the PostgreSQL database using SQLAlchemy."""
    db_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
    return create_engine(db_url)

def get_emissions_timeline(profil_id):
    """Fetch and combine emissions data over time for a specific user."""
    engine = connect_to_db()

    # Transport Emissions
    transport_query = f"""
        SELECT te.datedebut, SUM(te.co2emistransport) AS transport_co2 
        FROM TransportEmission te
        JOIN BEGES b ON te.BEGESId = b.BEGESId
        WHERE b.ProfilId = {profil_id}
        GROUP BY te.datedebut
        ORDER BY te.datedebut
    """
    transport_emissions = pd.read_sql_query(transport_query, con=engine)
    transport_emissions['datedebut'] = pd.to_datetime(transport_emissions['datedebut'])

    # Alimentation Emissions
    alimentation_query = f"""
        SELECT ae.datedebut, SUM(ae.co2emisalimentation) AS alimentation_co2 
        FROM AlimentationEmission ae
        JOIN BEGES b ON ae.BEGESId = b.BEGESId
        WHERE b.ProfilId = {profil_id}
        GROUP BY ae.datedebut
        ORDER BY ae.datedebut
    """
    alimentation_emissions = pd.read_sql_query(alimentation_query, con=engine)
    alimentation_emissions['datedebut'] = pd.to_datetime(alimentation_emissions['datedebut'])

    # Heating Emissions
    chauffage_query = f"""
        SELECT ce.datedebut, SUM(ce.co2emischauffage) AS chauffage_co2 
        FROM ChauffageEmission ce
        JOIN BEGES b ON ce.BEGESId = b.BEGESId
        WHERE b.ProfilId = {profil_id}
        GROUP BY ce.datedebut
        ORDER BY ce.datedebut
    """
    chauffage_emissions = pd.read_sql_query(chauffage_query, con=engine)
    chauffage_emissions['datedebut'] = pd.to_datetime(chauffage_emissions['datedebut'])

    # Combine data into a single DataFrame
    timeline = pd.DataFrame()
    timeline['date'] = transport_emissions['datedebut']
    timeline['transport_co2'] = transport_emissions['transport_co2']
    timeline = timeline.merge(alimentation_emissions, how='outer', left_on='date', right_on='datedebut')
    timeline = timeline.merge(chauffage_emissions, how='outer', left_on='date', right_on='datedebut')

    # Clean up columns
    timeline.drop(columns=['datedebut_x', 'datedebut_y'], inplace=True)
    timeline.rename(columns={'alimentation_co2': 'alimentation_co2', 'chauffage_co2': 'chauffage_co2'}, inplace=True)

    # Fill missing values with 0 for plotting
    timeline.fillna(0, inplace=True)

    # Sort by date
    timeline.sort_values(by='date', inplace=True)
    
    return timeline


def plot_emissions_timeline(timeline):
    """Plot the emissions timeline."""
    plt.figure(figsize=(14, 7))
    plt.plot(timeline['date'], timeline['transport_co2'], label='Transport CO2', marker='o')
    plt.plot(timeline['date'], timeline['alimentation_co2'], label='Alimentation CO2', marker='s')
    plt.plot(timeline['date'], timeline['chauffage_co2'], label='Heating CO2', marker='^')

    plt.title('User Emissions Evolution Over Time')
    plt.xlabel('Date')
    plt.ylabel('CO2 Emissions (kg)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    profil_id = 1  # Replace with the desired user ID
    timeline = get_emissions_timeline(profil_id)
    plot_emissions_timeline(timeline)


ValueError: You are trying to merge on datetime64[ns] and object columns for key 'date'. If you wish to proceed you should use pd.concat

In [16]:
!pip install matplotlib

  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.3-cp312-cp312-win_amd64.whl.metadata (168 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl (8.0 MB)
Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl (220 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.55.3-cp312-cp312-win_amd64.whl (2.2 MB)
Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl (71 kB)
Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl (2.6 MB)
Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)
